<a href="https://colab.research.google.com/github/svetamorag/publicLabs/blob/main/GenerativeAI_PaLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Code Sample

It is the code sample for working with Google Cloud PaLM2 API.

## Authenticate

In [ ]:
PROJECT_ID = "set here your project_id"  # @param {type:"string"}

from google.colab import auth as google_auth
google_auth.authenticate_user(project_id=PROJECT_ID)

## Dependencies

In [ ]:
!pip install google-cloud-aiplatform



In [ ]:
import vertexai
from vertexai.preview.language_models import TextGenerationModel
import pandas as pd
from google.cloud import storage
from google.cloud import translate_v3 as translate
import uuid
from io import StringIO

BUCKET='ccai_bucket_52788'
DATAFILE='GoogleStock_history_GOOGLE.csv'
REGION = 'us-central1'
PARENT="projects/"+PROJECT_ID+"/locations/"+REGION

##Build Prompt
The function buildPrompt() reads the data from the CSV file Google Stock history - GOOGLE.csv and stores it in a pandas data frame. It then creates a prompt that asks the user to provide the highest price, lowest price, and close price of the stock.

In [ ]:
def buildPrompt():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(BUCKET)
    blob = bucket.get_blob(DATAFILE)
    csvStringIO = StringIO(blob.download_as_text())
    df = pd.read_csv(csvStringIO)

    prompt = """Read this data for GOOG stock:
    """ + df.to_string()+"""
    and explain the status of the GOOGL stock based on the "close" column in the following format:
    GOOGLE stock information:
    The highest price was:
    The lowest price was:
    The close price in the end of the period was __.
    """
    return prompt

print(buildPrompt())

Read this data for GOOG stock:
                 Date    Open    High     Low   Close Change %      Volume
0   Jun. 16, 2023  126.70  126.70  123.79  124.06   -1.38%  56,699,200
1   Jun. 15, 2023  123.88  126.16  123.14  125.79    1.13%  24,517,071
2   Jun. 14, 2023  123.80  124.79  122.16  124.38   -0.04%  24,659,620
3   Jun. 13, 2023  125.65  125.86  123.85  124.43    0.06%  19,287,650
4   Jun. 12, 2023  123.40  124.75  122.35  124.35    1.20%  22,289,632
5   Jun. 09, 2023  122.56  124.29  122.42  122.87    0.16%  20,313,676
6   Jun. 08, 2023  122.59  123.73  122.01  122.67   -0.22%  24,814,996
7   Jun. 07, 2023  127.58  129.55  122.63  122.94   -3.89%  34,179,348
8   Jun. 06, 2023  126.60  128.88  125.97  127.91    1.01%  19,450,128
9   Jun. 05, 2023  124.61  127.99  124.38  126.63    1.12%  22,672,516
10  Jun. 02, 2023  124.49  126.75  124.35  125.23    0.69%  19,367,453
11  Jun. 01, 2023  123.50  125.04  123.30  124.37    0.81%  25,017,674
12   May 31, 2023  123.70  124.90  123.10 

##Work with PaLM2 API
This function uses the vertexai library to get a response from the PaLM 2 model. The prompt is first built using the "buildPrompt()" function, then the model is initialized and the response is returned.

In [ ]:
def getPaLM2Response():
    prompt=buildPrompt()
    vertexai.init(project=PROJECT_ID, location="us-central1")

    generation_model = TextGenerationModel.from_pretrained("text-bison@001")

    return generation_model.predict(prompt=prompt, max_output_tokens=256).text

getPaLM2Response()

'GOOGLE stock information:\n    The highest price was 128.88.\n    The lowest price was 104.50.\n    The close price in the end of the period was 124.06.'

##Translate text
This function translates a string text to a language specified by target.

In [ ]:

def translateText(target: str, text: str) -> dict:
    # Replace 'your-project-id' with your actual GCP project ID

    client = translate.TranslationServiceClient()
    response = client.translate_text(
        request={
            "parent": PARENT,
            "contents": [text],
            "target_language_code": target,
        }
    )

    result = response.translations[0].translated_text
    return result

translateText('he', getPaLM2Response())



'מידע על מניית GOOGLE: המחיר הגבוה ביותר היה 128.88. המחיר הנמוך ביותר היה 104.50. מחיר הסגירה בסוף התקופה עמד על 124.06.'

> by Sveta Morag (@svetam)




